In [1]:
import os
import numpy as np

import torch

from torch import Tensor
from torch import utils, nn, optim, cuda
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

import lightning as L
import torch.nn.functional as F

# https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
torch.set_float32_matmul_precision("high")

device = "cuda" if cuda.is_available() else "cpu"
device

'cuda'

In [41]:
from torchmetrics import Accuracy


class MNISTClassifier(L.LightningModule):
    def __init__(self):
        super().__init__()

        # 28 x 28 x 1
        self.lin = nn.Sequential(
            nn.Linear(28 * 28, 128, device=device),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, 64, device=device),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 10, device=device),
            nn.Softmax(dim=1),
        )
        self.accuracy = Accuracy(task="binary")

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1)
        return [optimizer], [lr_scheduler]

    def training_step(self, batch, batch_idx):
        x, y = batch

        x = x.view(-1, 28 * 28)
        x = self.lin(x)

        loss = F.cross_entropy(x, y)
        acc = self.accuracy(x, y)

        self.log("train_loss", loss, on_epoch=True, prog_bar=True, logger=True)
        self.log("train_acc", acc, on_epoch=True, prog_bar=True)

        return loss

    # def training_epoch_end(self, outputs) -> None:
    #     loss = sum(output['loss'] for output in outputs) / len(outputs)
    #     print(loss)

    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        x, y = batch

        x = x.view(-1, 28 * 28)
        x = self.lin(x)

        loss = F.cross_entropy(x, y)
        acc = self.accuracy(x, y)

        self.log("val_loss", loss, on_epoch=True, prog_bar=True, logger=True)
        self.log("val_acc", acc, on_epoch=True, prog_bar=True)

    def test_step(self, batch, batch_idx):
        # this is the test loop
        x, y = batch

        x = x.view(-1, 28 * 28)
        x = self.lin(x)

        test_loss = F.cross_entropy(x, y)
        self.log("val_loss", test_loss)

    def predict_step(self, batch, batch_idx):
        x, _ = batch
        pred = self.lin(x)
        return pred

    def forward(self, x: Tensor) -> Tensor:
        """Used for model(x) inference"""
        with torch.no_grad():
            x = x.view(-1, 28 * 28)
            x = self.lin(x)
            return x

In [42]:
from pathlib import Path


root_dir = Path(os.getcwd()).parent
root_dir.absolute()

PosixPath('/home/solan/repos/iu/Course 3/Fall Semester/PMLDL/Assignments/1/repo')

In [43]:
def target_transform(x: int) -> Tensor:
    temp = np.zeros(10)
    temp[x] = 1

    return torch.from_numpy(temp)


train_dataset = MNIST(
    root_dir / "code" / "datasets",
    download=True,
    train=True,
    transform=ToTensor(),
    target_transform=target_transform,
)
val_dataset = MNIST(
    root_dir / "code" / "datasets",
    download=True,
    train=False,
    transform=ToTensor(),
    target_transform=target_transform,
)

# dataset.targets = F.one_hot(dataset.targets, num_classes=10)
train_loader = utils.data.DataLoader(train_dataset, batch_size=100, num_workers=10)
val_loader = utils.data.DataLoader(val_dataset, batch_size=100, num_workers=10)

In [44]:
model = MNISTClassifier()
trainer = L.Trainer(
    # limit_train_batches=100,
    # limit_val_batches=100,
    max_epochs=5,
    enable_checkpointing=True,
    enable_model_summary=True,
    default_root_dir=root_dir / "models",
)
trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type           | Params | Mode 
----------------------------------------------------
0 | lin      | Sequential     | 109 K  | train
1 | accuracy | BinaryAccuracy | 0      | train
----------------------------------------------------
109 K     Trainable params
0         Non-trainable params
109 K     Total params
0.439     Total estimated model params size (MB)
10        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Epoch 4: 100%|██████████| 600/600 [00:11<00:00, 51.02it/s, v_num=10, train_loss_step=1.480, train_acc_step=0.996, val_loss=1.500, val_acc=0.995, train_loss_epoch=1.490, train_acc_epoch=0.995]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 600/600 [00:11<00:00, 50.87it/s, v_num=10, train_loss_step=1.480, train_acc_step=0.996, val_loss=1.500, val_acc=0.995, train_loss_epoch=1.490, train_acc_epoch=0.995]


In [45]:
# trainer.test(model=model, dataloaders=val_loader)
trainer.validate(model=model, dataloaders=val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation DataLoader 0: 100%|██████████| 100/100 [00:01<00:00, 91.40it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.9945998191833496     │
│         val_loss          │    1.4960873269677162     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 1.4960873269677162, 'val_acc': 0.9945998191833496}]

In [46]:
# predict_loader =
from PIL import Image

with Image.open(root_dir / "image.png") as file:
    x = np.array(file, dtype=np.float32)

x = torch.from_numpy(x)

model.freeze()
preds: Tensor = model(x)
# print(preds.shape)

preds.argmax(dim=1).item()

5

In [47]:
torch.save(model.state_dict(), root_dir / "models" / "state_dict.pt")

In [48]:
torch.save(model, root_dir / "models" / "model.pt")

In [10]:
ckpt_path = (
    root_dir
    / "models"
    / "lightning_logs"
    / "version_0"
    / "checkpoints"
    / "epoch=4-step=500.ckpt"
)
loaded_model = MNISTClassifier.load_from_checkpoint(checkpoint_path=ckpt_path)
loaded_model.to(device)

MNISTClassifier(
  (lin): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
    (5): Softmax(dim=1)
  )
)

In [11]:
x = x.to(device)

loaded_model.freeze()
preds = loaded_model(x)

preds.argmax(dim=1).item()

<built-in method type of Tensor object at 0x7e1d52f12df0>


6

In [16]:
torch.serialization.add_safe_globals(
    [MNISTClassifier, set, nn.Sequential, nn.Linear, nn.ReLU, nn.Softmax, Attri]
)

In [19]:
other_loaded_model = MNISTClassifier()
state_dict = torch.load(root_dir / "models" / "state_dict.pt", weights_only=True)
other_loaded_model.load_state_dict(state_dict)
other_loaded_model.to(device)

MNISTClassifier(
  (lin): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
    (5): Softmax(dim=1)
  )
)